# IBM Capstone Project1

In [5]:
##import packages
import pandas as pd
import numpy as np
import requests

### Install wikipedia package to load data 
### Then reserve it in dataframe

In [6]:
!pip install wikipedia
import wikipedia as wp
 
#Get the html source
html = wp.page("List_of_postal_codes_of_Canada:_M").html().encode("UTF-8")
df = pd.read_html(html)[0]

print (df)

    Postcode           Borough          Neighbourhood
0        M1A      Not assigned           Not assigned
1        M2A      Not assigned           Not assigned
2        M3A        North York              Parkwoods
3        M4A        North York       Victoria Village
4        M5A  Downtown Toronto           Harbourfront
..       ...               ...                    ...
282      M8Z         Etobicoke              Mimico NW
283      M8Z         Etobicoke     The Queensway West
284      M8Z         Etobicoke  Royal York South West
285      M8Z         Etobicoke         South of Bloor
286      M9Z      Not assigned           Not assigned

[287 rows x 3 columns]


In [7]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


####  Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [9]:
data1=df[~(df['Borough']=='Not assigned')]

In [10]:
data1

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


######  More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [11]:
data2=data1.groupby('Postcode')

In [12]:
Borough=data2.Borough.unique()

In [13]:
data2=data1.groupby('Postcode').agg(','.join)

In [14]:
data2.Borough=Borough

In [15]:
data2

,Borough,Neighbourhood
Postcode,,
M1B,[Scarborough],"Rouge,Malvern"
M1C,[Scarborough],"Highland Creek,Rouge Hill,Port Union"
M1E,[Scarborough],"Guildwood,Morningside,West Hill"
M1G,[Scarborough],Woburn
M1H,[Scarborough],Cedarbrae
...,...,...
M9N,[York],Weston
M9P,[Etobicoke],Westmount
M9R,[Etobicoke],"Kingsview Village,Martin Grove Gardens,Richvie..."


In [16]:
## reset index
data2.reset_index(inplace=True)

In [17]:
data2

,Postcode,Borough,Neighbourhood
0,M1B,[Scarborough],"Rouge,Malvern"
1,M1C,[Scarborough],"Highland Creek,Rouge Hill,Port Union"
2,M1E,[Scarborough],"Guildwood,Morningside,West Hill"
3,M1G,[Scarborough],Woburn
4,M1H,[Scarborough],Cedarbrae
...,...,...,...
98,M9N,[York],Weston
99,M9P,[Etobicoke],Westmount
100,M9R,[Etobicoke],"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,[Etobicoke],"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


###### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [18]:
data2.loc[data2[data2.Neighbourhood=='Not assigned'].index,'Neighbourhood']=data2[data2.Neighbourhood=='Not assigned']['Borough']

In [19]:
## exame if there is Not assigned Neighbourhood
data2[data2.Neighbourhood=='Not assigned']

,Postcode,Borough,Neighbourhood


####  In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [20]:
data2.shape

(103, 3)

In [21]:
data2.shape[0]

103

#### load location file

In [22]:
new=pd.read_csv('http://cocl.us/Geospatial_data')

In [23]:
## change column names to merge data
new.columns=['Postcode','Latitude','Longitude']

In [24]:
Toronto=data2.merge(new)

In [25]:
Toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,[Scarborough],"Rouge,Malvern",43.806686,-79.194353
1,M1C,[Scarborough],"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,[Scarborough],"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,[Scarborough],Woburn,43.770992,-79.216917
4,M1H,[Scarborough],Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,[York],Weston,43.706876,-79.518188
99,M9P,[Etobicoke],Westmount,43.696319,-79.532242
100,M9R,[Etobicoke],"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,[Etobicoke],"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


In [26]:
for i in range(Toronto.shape[0]):
    Toronto.loc[i,'Borough']=Toronto.loc[i,'Borough'][0]

In [27]:
Toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


In [28]:
Toronto=Toronto.astype({'Latitude':'object','Longitude':'object'})

In [29]:
Toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
...,...,...,...,...,...
98,M9N,York,Weston,43.7069,-79.5182
99,M9P,Etobicoke,Westmount,43.6963,-79.5322
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.6889,-79.5547
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.7394,-79.5884


### Thank you for reviewing my IBM Capstone project1 

In [30]:
Toronto.Borough.unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke', "Queen's Park"], dtype=object)

In [31]:
res=Toronto[Toronto['Borough'].str.contains("Toronto")]

In [32]:
res.reset_index(inplace=True)

In [33]:
res.drop('index',1)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.6764,-79.293
1,M4K,East Toronto,"The Danforth West,Riverdale",43.6796,-79.3522
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.669,-79.3156
3,M4M,East Toronto,Studio District,43.6595,-79.3409
4,M4N,Central Toronto,Lawrence Park,43.728,-79.3888
5,M4P,Central Toronto,Davisville North,43.7128,-79.3902
6,M4R,Central Toronto,North Toronto West,43.7154,-79.4057
7,M4S,Central Toronto,Davisville,43.7043,-79.3888
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.6896,-79.3832
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.6864,-79.4


In [34]:
res.shape

(39, 6)

In [35]:
print(res.Borough.unique())

['East Toronto' 'Central Toronto' 'Downtown Toronto' 'West Toronto']


In [36]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium

import folium # map rendering library

In [37]:
latitude=res['Latitude'][1]
longitude=res['Longitude'][1]
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

In [38]:
# add markers to map
for lat, lng, borough, neighborhood in zip(res['Latitude'], res['Longitude'], res['Borough'], res['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [39]:
CLIENT_ID = 'XXZ4ET0T2B0XK2G25I5GRI32SC1B512J5FKOYDCP5IFGJ4PC' # your Foursquare ID
CLIENT_SECRET = 'ZEVV5MGYRQADZQXW440KOBJLGZAFMKBTLFYOVSUJXFDUO4VC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XXZ4ET0T2B0XK2G25I5GRI32SC1B512J5FKOYDCP5IFGJ4PC
CLIENT_SECRET:ZEVV5MGYRQADZQXW440KOBJLGZAFMKBTLFYOVSUJXFDUO4VC


## Let's try to explore a beighborhood

In [43]:
neighborhood_latitude=res['Latitude'][0]
neighborhood_longitude=res['Longitude'][0]
print('The borough {},neighborhood {} with location '.format(res['Borough'][0],res['Neighbourhood'][0])+str(neighborhood_latitude)+','+str(neighborhood_latitude))

The borough East Toronto,neighborhood The Beaches with location 43.67635739999999,43.67635739999999


In [44]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
neighborhood_json = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = neighborhood_json['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [46]:
toronto_venues_df = getNearbyVenues(names=res['Neighbourhood'],
                                   latitudes=res['Latitude'],
                                   longitudes=res['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede

In [47]:
print(toronto_venues_df.shape)
toronto_venues_df.head()

(1698, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [48]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues_df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.04
2       Steakhouse  0.04
3   Cosmetics Shop  0.03
4  Thai Restaurant  0.03


----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.05
2        Beer Bar  0.04
3  Farmers Market  0.04
4            Café  0.04


----Brockton,Exhibition Place,Parkdale Village----
            venue  freq
0     Coffee Shop  0.09
1            Café  0.09
2  Breakfast Spot  0.09
3          Bakery  0.05
4       Nightclub  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0           Yoga Studio  0.06
1         Auto Workshop  0.06
2            Comic Shop  0.06
3                  Park  0.06
4  Gym / Fitness Center  0.06


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.21
1    Airport Lounge  0.14
2  Airport Terminal  0.14
3

In [50]:
#method to sort venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [58]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Restaurant,Asian Restaurant,Bakery,Bar,Cosmetics Shop,Thai Restaurant,Sushi Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Café,Beer Bar,Bakery,Seafood Restaurant,Cheese Shop,Steakhouse,Shopping Mall
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Breakfast Spot,Café,Climbing Gym,Stadium,Bar,Bakery,Restaurant,Intersection,Pet Store
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Park,Restaurant,Fast Food Restaurant,Farmers Market,Auto Workshop,Spa,Pizza Place,Burrito Place,Light Rail Station
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Boutique,Airport,Airport Food Court,Sculpture Garden,Bar,Harbor / Marina


In [59]:
# set number of clusters
kclusters = 4
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = res.copy()
toronto_merged.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()# check the last columns!

,index,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,37,M4E,East Toronto,The Beaches,43.6764,-79.293,0,Pub,Trail,Health Food Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store
1,41,M4K,East Toronto,"The Danforth West,Riverdale",43.6796,-79.3522,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Furniture / Home Store,Diner,Sports Bar,Liquor Store,Dessert Shop
2,42,M4L,East Toronto,"The Beaches West,India Bazaar",43.669,-79.3156,0,Sandwich Place,Fast Food Restaurant,Fish & Chips Shop,Steakhouse,Pub,Sushi Restaurant,Pizza Place,Food & Drink Shop,Ice Cream Shop,Pet Store
3,43,M4M,East Toronto,Studio District,43.6595,-79.3409,0,Café,Coffee Shop,Gastropub,Brewery,Bakery,Italian Restaurant,American Restaurant,Comfort Food Restaurant,Sandwich Place,Cheese Shop
4,44,M4N,Central Toronto,Lawrence Park,43.728,-79.3888,0,Park,Lawyer,Bus Line,Swim School,Women's Store,Deli / Bodega,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [60]:

# create map
map_clusters = folium.Map(location=[res["Latitude"][0], res["Longitude"][0]], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters